<a href="https://colab.research.google.com/github/parastooAflaki/deeplearning/blob/master/simple2layerNN_tensorflow_v1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

In [0]:
def linearfunc():
  X = tf.constant(np.random.randn(3,1),name = "X")
  W = tf.constant(np.random.randn(4,3), name = "W")
  b = tf.constant(np.random.randn(4,1),name="b")
  Y = tf.add(tf.matmul(W,X),b)

  sess = tf.Session()
  result = sess.run(Y)
  sess.close()
  return result

In [15]:
print(str(linearfunc()))

[[-1.08510042]
 [ 0.39044508]
 [ 1.80742228]
 [-3.33794844]]


In [0]:
def create_placeholders (n_x , n_y):
  #nx shape of image
  #ny number of classes
  X = tf.placeholder(dtype=float , shape = (n_x , None) , name = "Placeholder_1")
  Y = tf.placeholder(dtype=float , shape = (n_y , None) , name = "Placeholder_2")
  return X, Y

In [6]:
X, Y = create_placeholders(12288, 6)
print ("X = " + str(X))
print ("Y = " + str(Y))

X = Tensor("Placeholder_1:0", shape=(12288, ?), dtype=float32)
Y = Tensor("Placeholder_2:0", shape=(6, ?), dtype=float32)


In [0]:
def initialize_parameters():
 
  W1 = tf.get_variable(name = "W1" , shape=[25,12288] , initializer=tf.contrib.layers.xavier_initializer())
  b1 = tf.get_variable("b1" , shape= [25 , 1] , initializer=tf.zeros_initializer())
  W2 = tf.get_variable( "W2" , shape=[12,25] , initializer=tf.contrib.layers.xavier_initializer())
  b2 = tf.get_variable("b2" , shape= [12 , 1] , initializer=tf.zeros_initializer())
  W3 = tf.get_variable("W3" , shape=[6,12] , initializer=tf.contrib.layers.xavier_initializer())
  b3 = tf.get_variable("b3" , shape= [6 , 1] , initializer=tf.zeros_initializer())

  parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
  return parameters


In [26]:
tf.reset_default_graph()
with tf.Session() as sess:  
    parameters = initialize_parameters()
    print("W1 = " + str(parameters["W1"]))
    print("b1 = " + str(parameters["b1"]))
    print("W2 = " + str(parameters["W2"]))
    print("b2 = " + str(parameters["b2"]))

W1 = <tf.Variable 'W1:0' shape=(25, 12288) dtype=float32_ref>
b1 = <tf.Variable 'b1:0' shape=(25, 1) dtype=float32_ref>
W2 = <tf.Variable 'W2:0' shape=(12, 25) dtype=float32_ref>
b2 = <tf.Variable 'b2:0' shape=(12, 1) dtype=float32_ref>


In [0]:
def forward_propagation(X, parameters):
  #the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    Z1 = tf.add(tf.matmul(W1 , X) , b1)                                              # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                              # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2 , A1) , b2)                                              # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                              # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3 , A2) , b3) 
    return Z3

In [32]:
tf.reset_default_graph()
with tf.Session() as sess:
    X, Y = create_placeholders(12288, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    print("Z3 = " + str(Z3))

Z3 = Tensor("Add_2:0", shape=(6, ?), dtype=float32)


In [0]:
def compute_cost(Z3, Y):
 
    
#   Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
#   Y -- "true" labels vector placeholder, same shape as Z3

#    cost - Tensor of the cost function
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels , logits=logits) )
 
    return cost


 ''' 
    It is important to know that the "logits" and "labels" inputs of tf.nn.softmax_cross_entropy_with_logits are expected to be of shape (number of examples, num_classes). We have thus transposed Z3 and Y.
    Besides, tf.reduce_mean basically does the summation over the examples.
  '''


After you compute the cost function. You will create an "optimizer" object. You have to call this object along with the cost when running the tf.session. When called, it will perform an optimization on the given cost with the chosen method and learning rate


In [0]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    ops.reset_default_graph() 
    (n_x, m) = X_train.shape
    n_y = Y_train.shape[0] 
    costs = []  

    X, Y = create_placeholders(n_x , n_y)
    parameters = initialize_parameters();
    Z3 = forward_propagation(X , parameters= parameters)
    cost = compute_cost(Z3 , Y)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0001 ).minimize(cost);

    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            for minibatch in minibatches:
                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                _ , minibatch_cost = sess.run([optimizer , cost] , feed_dict= {X : minibatch_X , Y : minibatch_Y})
                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
            parameters = sess.run(parameters)
            print ("Parameters have been trained!")
      
      # lets save the parameters in a variable
            parameters = sess.run(parameters)
            print ("Parameters have been trained!")
       
        # Calculate the correct predictions
            correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

            print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
            print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters

